In [ ]:
from rag.engines.GithubEngine import GithubEngine

query_engine = GithubEngine()
engine_ref = query_engine._create_nonstreaming_engine()
# response = query_engine.answer("What does Batch.py do? and also, how many preprocessed pages does the book Jip&Janneke have?")
# print(response)


In [ ]:
response = engine_ref.query("Explain the code websockets code")
response

In [ ]:
print(response)

In [ ]:
import logging
import sys
import pickle
import os
import nest_asyncio

nest_asyncio.apply()
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))



In [ ]:
from llama_index import download_loader, GPTVectorStoreIndex
download_loader("GithubRepositoryReader")


from llama_index.readers.github import GithubRepositoryReader,GithubClient
github_client = GithubClient(os.getenv("GITHUB_TOKEN"))

from llama_index.core import Settings
from llama_index.llms.ollama import Ollama

Settings.llm = Ollama(model="llama3.2:3b",request_timeout=120.0,context_window=8000)


In [ ]:
class Repo:
    def __init__(self, **kwargs):
        self.owner = kwargs['owner']
        self.name = kwargs['name']
        self.inc_dir = kwargs['inc_dir']
        self.inc_ext = kwargs['inc_ext']
        self.docs = None
        self.index = None
        self.query_engine = None

        try:
            self.branch = kwargs['branch']
        except KeyError:
            try:
                self.commit_sha = kwargs['commit_sha']
            except KeyError:
                raise KeyError("Must specify a branch or sha_commit")

    def load(self, force_reload=False, verbose=True, 
             concurrent_requests=10,
             persist_dir='./data/gitrepo'):
        self.pkl_file = f"{persist_dir}/{self.owner}_{self.name}.pkl"

        if not os.path.exists(persist_dir):
            os.makedirs(persist_dir)

        if force_reload:
            self._reload(verbose, concurrent_requests)
        else: 
            try:
                self._load_pkl()
            except:
                self._reload(verbose, concurrent_requests)

    def _reload(self, verbose, concurrent_requests):
        if os.path.exists(self.pkl_file):
            os.remove(self.pkl_file)
        self._load_gh(verbose, concurrent_requests)
        self._persist_pkl()

    def _persist_pkl(self):
        with open(self.pkl_file, "wb") as f:
            pickle.dump(self.docs, f)

    def _load_pkl(self):
        with open(self.pkl_file, "rb") as f:
            self.docs = pickle.load(f)

    def _load_gh(self, verbose, concurrent_requests):
        loader = GithubRepositoryReader(
            github_client,
            owner=self.owner,
            repo=self.name,
            filter_directories=(
                self.inc_dir,
                GithubRepositoryReader.FilterType.INCLUDE
                ),
            filter_file_extensions=(
                self.inc_ext, 
                GithubRepositoryReader.FilterType.INCLUDE
                ),
            verbose=verbose,
            concurrent_requests=concurrent_requests,
        )
        if self.branch:
            self.docs = loader.load_data(branch=self.branch)
        else: 
            self.docs = loader.load_data(commit_sha=self.commit_sha)

    def query(self, query):
        if self.query_engine is None:
            self._init_query()
        return str(self.query_engine.query(query))

    def _init_query(self):
        if self.docs is None: 
            self.load()
        self.index = GPTVectorStoreIndex.from_documents(self.docs)
        self.query_engine = self.index.as_query_engine()

In [ ]:
repo1 = {
    "owner": "OwaisNoor000",
    "name": "MySensayClone",
    "inc_dir": ["face", "rag","websockets"],
    "inc_ext": [".py"],
    "branch": "main",
}

In [ ]:
llama_repo = Repo(**repo1)
llama_repo.load(force_reload=True)

In [ ]:
llama_repo.query("How does this application utilize web sockets?")

In [1]:
from  supabasedb.MeetingRequestsDao import MeetingRequestsDao
from supabasedb.MeetingRequests import MeetingRequests

In [2]:

dao = MeetingRequestsDao()
meetingRequest = MeetingRequests("Testing")

In [3]:
dao.create(meetingRequest)

DEBUG: Operation failed
{'message': 'new row violates row-level security policy for table "MeetingRequests"', 'code': '42501', 'hint': None, 'details': None}


In [4]:
import supabase
supabase.auth.sign_in_with_password({
    "email":"mowaisnoor000@gmail",
    "password":"Pa$$w0rd=0wais"
})

AttributeError: module 'supabase' has no attribute 'auth'